### The architecture of the CNN used here is based on the paper "[Extreme Gradient Boost with CNN: A Deep Learning-Based Approach for Predicting Protein Subcellular Localization](https://link.springer.com/chapter/10.1007/978-981-16-6636-0_16#Tab1)" by Md. Ismail & Md. Nazrul Islam Mondal. We use modified one-hot encoded protein sequences as inputs.

In [1]:
# Run this code on Google Colab
!pip install xgboost

import tensorflow as tf

# Check if the GPU is available and output its name
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


Default GPU Device: /device:GPU:0


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.callbacks import EarlyStopping


import numpy as np

import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score

# Confirm that GPU is being used
print("TensorFlow version:", tf.__version__)
print(tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.15.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# The architecture is based on the paper mentioned in the title
def build_cnn(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)

    # Convolutional layers with different kernel sizes
    conv_layers = []
    kernel_sizes = [1, 3, 5, 9, 15, 21]
    for size in kernel_sizes:
        conv = Conv1D(filters=64, kernel_size=size, activation='relu', padding='same')(inputs)
        conv_layers.append(conv)

    # Concatenate all convolutional layers
    concatenated = concatenate(conv_layers, axis=-1)

    # Final convolutional layer
    final_conv = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(concatenated)

    # Max pooling layer
    pooled = MaxPooling1D(pool_size=5)(final_conv)

    # Dropout layer
    dropped = Dropout(rate=0.25)(pooled)

    # Flatten layer
    flatten = Flatten()(dropped)

    # Dense layer
    dense = Dense(64, activation='relu')(flatten)

    # Output layer for binary classification
    output = Dense(1, activation='sigmoid')(dense)

    # Create the model
    model = Model(inputs=inputs, outputs=output)
    return model

In [6]:
import gzip
import pickle


# Loading the compressed pickle file that contained one-hot encoding modified with the pssm scores
with gzip.open('cnn_input_data.pkl.gz', 'rb') as f:
    df = pickle.load(f)
print(df)


         ACC                                           Encoding  Label
0     O75439  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
1     Q2TBK2  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
2     Q5VY80  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
3     Q9BZM6  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
4     O75489  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
...      ...                                                ...    ...
2947  Q96CK0  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      1
2948  Q96CK0  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      1
2949  Q24JY4  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      1
2950  O43257  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      1
2951  Q8R331  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      1

[2951 rows x 3 columns]


In [7]:
# Prepare data
X = np.stack(df['Encoding'].values)
y = df['Label'].values.astype('int')  # Ensure labels are integers (0 or 1)

# K-Fold Cross-Validation setup
kf = KFold(n_splits=5, shuffle=True, ra ndom_state=50)
fold = 0
accuracies = []

for train_index, test_index in kf.split(X):
    fold += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Build and compile CNN model
    input_shape = (1000, 20)
    cnn_model = build_cnn(input_shape)
    cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)

    # Train CNN model
    history = cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

    # Extract features for XGBoost
    feature_layer_model = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
    X_train_features = feature_layer_model.predict(X_train)
    X_test_features = feature_layer_model.predict(X_test)

    # Train XGBoost on extracted features
    xgb_model = xgb.XGBClassifier()
    xgb_model.fit(X_train_features, y_train)

    # Predict using XGBoost
    y_pred_xgb = xgb_model.predict(X_test_features)

    # Calculate and store accuracy
    accuracy = accuracy_score(y_test, y_pred_xgb)
    accuracies.append(accuracy)
    print(f"Fold {fold}, Accuracy: {accuracy:.2%}")

# Average accuracy across all folds
average_accuracy = np.mean(accuracies)
print(f"Average Accuracy: {average_accuracy:.2%}")

Epoch 1/10
59/59 [==============================] - 5s 30ms/step - loss: 0.7509 - accuracy: 0.6822 - val_loss: 0.8076 - val_accuracy: 0.4407
Epoch 2/10
59/59 [==============================] - 1s 20ms/step - loss: 0.3581 - accuracy: 0.8284 - val_loss: 0.8823 - val_accuracy: 0.5678
Epoch 3/10
59/59 [==============================] - 1s 19ms/step - loss: 0.1752 - accuracy: 0.9359 - val_loss: 0.5178 - val_accuracy: 0.7627
Epoch 4/10
59/59 [==============================] - 1s 18ms/step - loss: 0.0806 - accuracy: 0.9778 - val_loss: 0.9896 - val_accuracy: 0.6674
Epoch 5/10
59/59 [==============================] - 1s 18ms/step - loss: 0.0663 - accuracy: 0.9815 - val_loss: 0.9486 - val_accuracy: 0.6758
Epoch 6/10
19/19 [==============================] - 0s 5ms/step
Fold 1, Accuracy: 93.06%
Epoch 1/10
59/59 [==============================] - 5s 34ms/step - loss: 0.8626 - accuracy: 0.6340 - val_loss: 0.8350 - val_accuracy: 0.1691
Epoch 2/10
59/59 [==============================] - 1s 19ms/step 

In [8]:
# Loading the testing data to do further prediction
with gzip.open('cnn_testing_data.pkl.gz', 'rb') as f:
    test_df = pickle.load(f)

print(test_df)

                                               Encoding  Label
0     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
1     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      1
2     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
3     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
4     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      1
...                                                 ...    ...
2709  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
2710  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
2711  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
2712  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
2713  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0

[2714 rows x 2 columns]


In [9]:
# Predict NLS on the testing set
X_test = np.stack(test_df['Encoding'].values)
y_test = test_df['Label'].values.astype('int')  # Ensure labels are integers (0 or 1)



X_test_features = feature_layer_model.predict(X_test)


# Predict using XGBoost
y_pred_xgb = xgb_model.predict(X_test_features)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy: {accuracy:.2%}")

85/85 [==============================] - 1s 9ms/step
Accuracy: 73.58%
